In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
import torch
import numpy as np
import cupy as cp
import time

# Import your custom modules
from laser_generator import LaserProfileGenerator
from thermomechanical import run_coupled_simulation

# --- CONFIGURATION ---
SIM_DURATION = 121.1  # Ensure this matches your .crs file duration
PROPERTIES_DIR = "."
GEOM_FILE = "thinwall.k"
OUTPUT_ZARR = "debug_stress.zarr"
cp.cuda.Device(2).use()



<CUDA Device 2>

In [ ]:
def run_debug():
    print("--- Starting Debug Simulation ---")
    
    # 1. Generate 10 random parameters (range [0, 1])
    random_params = np.random.rand(10)
    print(f"Random Parameters: \n{random_params}\n")

    # 2. Initialize the Laser Generator
    generator = LaserProfileGenerator(total_time=SIM_DURATION)
    
    # ---> CRITICAL FIX: Pre-calculate the profile before trying to preview it
    generator.generate_profile(random_params)
    
    # 3. Optional: Preview the generated power curve (first few steps)
    print("Previewing power profile...")
    test_times = [0.0, SIM_DURATION * 0.25, SIM_DURATION * 0.5, SIM_DURATION * 0.75]
    for t in test_times:
        # ---> CRITICAL FIX: Remove 'random_params' from this call
        p = generator.get_power_at_time(t)
        print(f"  t={t:.3f}s -> Power={p:.2f}W")
    print("")

    # 4. Run the Coupled Simulation
    start_wall = time.perf_counter()
    
    try:
        print(f"Executing Thermomechanical Solver on GPU {cp.cuda.Device().id}...")
        max_residual_stress = run_coupled_simulation(
            params=random_params,
            generator=generator,
            input_dir=PROPERTIES_DIR,
            geom_file=GEOM_FILE,
            output_path=OUTPUT_ZARR
        )
        
        end_wall = time.perf_counter()
        
        print("\n--- Simulation Success ---")
        print(f"Wall Time: {end_wall - start_wall:.2f} seconds")
        print(f"Max Residual Stress: {max_residual_stress:.2f} MPa")
        print(f"Results saved to: {os.path.abspath(OUTPUT_ZARR)}")
        
    except Exception as e:
        print(f"\n!!! Simulation Failed !!!")
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    run_debug()

--- Starting Debug Simulation ---
Random Parameters: 
[0.84500331 0.22474948 0.26236762 0.37768616 0.38276069 0.71744096
 0.87580005 0.82747826 0.38497844 0.17808692]

Previewing power profile...
  t=0.000s -> Power=55255.59W
  t=75.000s -> Power=325302.99W
  t=150.000s -> Power=182788.73W
  t=225.000s -> Power=-11670.87W

Executing Thermomechanical Solver on GPU 2...
DEBUG: Checking directory: /data/pxl1051/ded_dt_thermomechanical_solver/examples/ME 441
DEBUG: Files found in properties folder: ['wall.k', 'thermomechanical.py', 'debug.ipynb', 'laser_generator.py', 'toolpath.crs', '__pycache__', 'bo_loop.py', 'materials', 'debug_stress.zarr']


In [4]:
import zarr
import pyvista as pv
import numpy as np
import vtk
from gamma.simulator.gamma import domain_mgr

def convert_zarr_to_vtk_dynamic(zarr_dir, k_file, output_prefix, target_outputs=100):
    print(f"Reading mesh from: {k_file}")
    domain = domain_mgr(filename=k_file)
    
    # 1. Pull all mesh data down to the CPU
    nodes = domain.nodes.get() if hasattr(domain.nodes, 'get') else domain.nodes
    elements = domain.elements.get() if hasattr(domain.elements, 'get') else domain.elements
    
    # Pull the birth arrays to determine what is active
    node_birth = domain.node_birth.get() if hasattr(domain.node_birth, 'get') else domain.node_birth
    element_birth = domain.element_birth.get() if hasattr(domain.element_birth, 'get') else domain.element_birth
    
    print(f"Reading Zarr store from: {zarr_dir}")
    root = zarr.open(zarr_dir, mode='r')
    
    time_array = root['time'][:]
    num_steps = len(time_array)
    
    # Downsampling logic
    interval = max(1, num_steps // target_outputs)
    steps_to_process = list(range(0, num_steps, interval))
    if steps_to_process[-1] != num_steps - 1:
        steps_to_process.append(num_steps - 1)
        
    print(f"Downsampled Zarr to {len(steps_to_process)} frames.")

    for step in steps_to_process:
        current_time = time_array[step]
        print(f"Processing step {step}/{num_steps - 1} (Sim Time: {current_time:.4f}s)...")
        
        # --- DYNAMIC MESH LOGIC ---
        # 1. Create masks for nodes and elements born before the current time
        active_n_mask = node_birth <= current_time
        active_e_mask = element_birth <= current_time
        
        # 2. Extract active nodes and displacements
        U = root['U'][step]
        U_active = U[active_n_mask]
        points_active = nodes[active_n_mask] + 5 * U_active 
        
        # 3. Remap Element Connectivity (CRITICAL)
        global_elements_active = elements[active_e_mask]
        
        # Create a mapping array from Global ID -> Local ID
        node_map = np.full(len(nodes), -1)
        node_map[active_n_mask] = np.arange(np.sum(active_n_mask))
        
        # Apply the map to the active elements
        local_elements_active = node_map[global_elements_active]
        
        # Fast vectorized way to build PyVista cells: [8, p1, p2, p3, p4, p5, p6, p7, p8]
        active_cells = np.hstack((np.full((len(local_elements_active), 1), 8), local_elements_active)).flatten()
        active_cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(local_elements_active))
        
        # Initialize Grid
        grid = pv.UnstructuredGrid(active_cells, active_cell_type, points_active)
        
        # Assign Point Data (only to active nodes)
        grid.point_data['temp'] = root['temperature'][step][active_n_mask]
        grid.point_data['U1'] = U_active[:, 0]
        grid.point_data['U2'] = U_active[:, 1]
        grid.point_data['U3'] = U_active[:, 2]

        # Assign Cell Data (only to active elements)
        S_ip = root['stress'][step]
        S_ip_active = S_ip[active_e_mask]
        S_avg = np.mean(S_ip_active, axis=1) 
        
        grid.cell_data['S11'] = S_avg[:, 0]
        grid.cell_data['S22'] = S_avg[:, 1]
        grid.cell_data['S33'] = S_avg[:, 2]
        grid.cell_data['S12'] = S_avg[:, 3]
        grid.cell_data['S23'] = S_avg[:, 4]
        grid.cell_data['S13'] = S_avg[:, 5]

        Sv = np.sqrt(0.5 * ((S_avg[:,0]-S_avg[:,1])**2 + (S_avg[:,1]-S_avg[:,2])**2 + (S_avg[:,2]-S_avg[:,0])**2 + 6*(S_avg[:,3]**2 + S_avg[:,4]**2 + S_avg[:,5]**2)))
        grid.cell_data['S_von'] = Sv

        grid.save(f"output/{output_prefix}_{step}.vtk")
        
    print("Done!")

# Execution
convert_zarr_to_vtk_dynamic('./debug_stress.zarr', 'thinwall.k', 'output_mesh', target_outputs=1500)

Reading mesh from: thinwall.k
Time of reading input files: 0.26503825187683105
Time of calculating critical timestep: 0.03282451629638672
Time of reading and interpolating toolpath: 0.08600258827209473
Number of nodes: 23249
Number of elements: 19076
Number of time-steps: 242200
Time of generating surface: 0.10130977630615234
Reading Zarr store from: ./debug_stress.zarr
Downsampled Zarr to 1607 frames.
Processing step 0/1606 (Sim Time: 0.1000s)...
Processing step 1/1606 (Sim Time: 0.1200s)...
Processing step 2/1606 (Sim Time: 0.1400s)...
Processing step 3/1606 (Sim Time: 0.1600s)...
Processing step 4/1606 (Sim Time: 0.1800s)...
Processing step 5/1606 (Sim Time: 0.2000s)...
Processing step 6/1606 (Sim Time: 0.2200s)...
Processing step 7/1606 (Sim Time: 0.2400s)...
Processing step 8/1606 (Sim Time: 0.2600s)...
Processing step 9/1606 (Sim Time: 0.2800s)...
Processing step 10/1606 (Sim Time: 0.3000s)...
Processing step 11/1606 (Sim Time: 0.3200s)...
Processing step 12/1606 (Sim Time: 0.340

In [7]:
import zarr
import pyvista as pv
import numpy as np
import vtk
from gamma.simulator.gamma import domain_mgr
def evaluate_objectives_from_zarr(zarr_dir, k_file):
    print(f"\n--- EVALUATING OBJECTIVES FROM ZARR ---")
    print(f"Reading mesh from: {k_file}")
    domain = domain_mgr(filename=k_file)
    
    # Extract birth arrays to CPU
    element_birth = domain.element_birth.get() if hasattr(domain.element_birth, 'get') else domain.element_birth
    node_birth = domain.node_birth.get() if hasattr(domain.node_birth, 'get') else domain.node_birth
    
    print(f"Reading Zarr store from: {zarr_dir}")
    root = zarr.open(zarr_dir, mode='r')
    
    # Get the time array and find the final simulation time
    time_array = np.array(root['time'][:]).flatten()
    final_time = time_array[-1]
    
    # Determine which nodes and elements physically exist at the end
    deposit_e_mask = (element_birth > 1e-5) & (element_birth <= final_time)
    deposit_n_mask = (node_birth > 1e-5) & (node_birth <= final_time)
    
    print(f"Final Sim Time: {final_time:.4f}s")
    print(f"Deposited Elements: {np.sum(deposit_e_mask)} / {len(element_birth)}")
    print(f"Deposited Nodes: {np.sum(deposit_n_mask)} / {len(node_birth)}")

    # ==========================================
    # 1. RESIDUAL STRESS OBJECTIVES
    # ==========================================
    print("Calculating Residual Stress Objectives...")
    # Get the stress state from the final recorded timestep
    S_final_all = np.array(root['stress'][-1]) # Shape: [n_elements, n_ip, 6]
    
    # Filter to only the active elements
    S_final = S_final_all[deposit_e_mask]
    
    # Calculate Von Mises on all integration points (Exact simulator logic)
    S_vm = np.sqrt(0.5 * ((S_final[:,:,0]-S_final[:,:,1])**2 + 
                          (S_final[:,:,1]-S_final[:,:,2])**2 + 
                          (S_final[:,:,2]-S_final[:,:,0])**2 + 
                          6*(S_final[:,:,3]**2 + S_final[:,:,4]**2 + S_final[:,:,5]**2)))
    
    max_residual_stress = float(np.max(S_vm))
    avg_residual_stress = float(np.mean(S_vm))

    # ==========================================
    # 2. HEAT TREATMENT TIME OBJECTIVES
    # ==========================================
    print("Calculating Heat Treatment Objectives...")
    T_MIN_HT = 654.0 + 273.15  # IN718 Min (Kelvin)
    T_MAX_HT = 857.0 + 273.15  # IN718 Max (Kelvin)
    
    # Load the entire temperature history [time_steps, total_nodes]
    T_history = np.array(root['temperature'][:])
    
    # Boolean mask for the critical temperature window
    in_range = (T_history >= T_MIN_HT) & (T_history <= T_MAX_HT)
    t_history_2d = time_array[:, np.newaxis]
    
    # Find LAST time in range
    t_max_masked = np.where(in_range, t_history_2d, -1.0)
    last_time_in_range = np.max(t_max_masked, axis=0)
    
    # Find FIRST time in range
    t_min_masked = np.where(in_range, t_history_2d, np.inf)
    first_time_in_range = np.min(t_min_masked, axis=0)
    
    # Calculate duration
    valid_nodes = last_time_in_range >= first_time_in_range
    ht_durations = np.zeros(len(node_birth), dtype=np.float64)
    ht_durations[valid_nodes] = last_time_in_range[valid_nodes] - first_time_in_range[valid_nodes]
    
    # Isolate physically active nodes
    active_ht_durations = ht_durations[deposit_n_mask]
    
    avg_heat_treatment_time = float(np.mean(active_ht_durations))
    min_heat_treatment_time = float(np.min(active_ht_durations))

    # --- PRINT RESULTS ---
    print("\n" + "="*40)
    print("FINAL OBJECTIVE EVALUATION")
    print("="*40)
    print(f"Max Residual Stress: {max_residual_stress:.2f} MPa")
    print(f"Avg Residual Stress: {avg_residual_stress:.2f} MPa")
    print(f"Avg Heat Treatment:  {avg_heat_treatment_time:.4f} seconds")
    print(f"Min Heat Treatment:  {min_heat_treatment_time:.4f} seconds")
    print("="*40 + "\n")

    return max_residual_stress, avg_residual_stress, avg_heat_treatment_time, min_heat_treatment_time

# Execute the evaluator
evaluate_objectives_from_zarr('./debug_stress.zarr', 'thinwall.k')


--- EVALUATING OBJECTIVES FROM ZARR ---
Reading mesh from: thinwall.k
Time of reading input files: 0.8619811534881592
Time of calculating critical timestep: 0.035711050033569336
Time of reading and interpolating toolpath: 0.2693331241607666
Number of nodes: 23249
Number of elements: 19076
Number of time-steps: 242200
Time of generating surface: 0.32737278938293457
Reading Zarr store from: ./debug_stress.zarr
Final Sim Time: 121.0805s
Deposited Elements: 14000 / 19076
Deposited Nodes: 17040 / 23249
Calculating Residual Stress Objectives...
Calculating Heat Treatment Objectives...

FINAL OBJECTIVE EVALUATION
Max Residual Stress: 1345.90 MPa
Avg Residual Stress: 411.98 MPa
Avg Heat Treatment:  49.3850 seconds
Min Heat Treatment:  0.7000 seconds



(1345.8984375, 411.9844970703125, 49.38504933057257, 0.7000045776367188)